In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings 
warnings.filterwarnings('ignore')


In [2]:
# load the dataset
df=pd.read_csv('Train.csv')
df.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
df.sample(10)

,id,qid1,qid2,question1,question2,is_duplicate
45394,45394,81363,81364,Why are almost all answers anonymous in Quora?,What are the anonymous answers on Quora?,0
327098,327098,426862,453489,Why does an English Cocker Spaniel shed so much?,Why do English Cocker Spaniels shed so much?,1
123216,123216,199270,199271,Which are the most visited temples of Chhattis...,Which are the most visited Temples in Chhattis...,1
120443,120443,95048,195351,Why does yellow mustard relieve a burn?,What is the difference between Dijon mustard a...,0
296161,296161,418321,418322,Was jerry more jerk than tom in Tom and jerry?,Why was jerry the real jerk in tom and jerry?,1
242664,242664,354919,354920,How do I prevent uglifyjs from converting numb...,Should schools teach cooking?,0
265366,265366,294118,382321,Why does anxiety cause irrational fears? What ...,What irrational fears do you have?,0
46731,46731,83552,83553,What is the advantage for European asset manag...,What is the best model for managing IT product...,0
149231,149231,235121,186237,What is the value of 1 dollar in indian rupees?,Do Americans see $1 the same way Indians see 1...,0
82065,82065,139246,139247,How tough is the GATE exam?,How tough is GATE Examination?,1


In [4]:
df.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


# FeatureEngineering the text

In [5]:
from tqdm import tqdm

class FeatureEngineering:
    def __init__(self, dataframe):
        self.df = dataframe
        tqdm.pandas()
        self.calculate_frequency_features()
        self.calculate_length_and_word_features()

    def calculate_frequency_features(self):
        self.df['freq_qid1'] = self.df['qid1'].map(self.df['qid1'].value_counts())
        self.df['freq_qid2'] = self.df['qid2'].map(self.df['qid2'].value_counts())
        self.df['frq_q1+q2'] = self.df['freq_qid1'] + self.df['freq_qid2']
        self.df['frq_q1-q2'] = abs(self.df['freq_qid1'] - self.df['freq_qid2'])

    def calculate_length_and_word_features(self):
        self.df['q1len'] = self.df['question1'].str.len()
        self.df['q2len'] = self.df['question2'].str.len()
        self.df['q1_n_words'] = self.df['question1'].progress_apply(lambda row: len(str(row).split()))
        self.df['q2_n_words'] = self.df['question2'].progress_apply(lambda row: len(str(row).split()))
        self.df['word_common'] = self.df.progress_apply(self.word_common, axis=1)
        self.df['word_share'] = self.df.progress_apply(self.word_share, axis=1)
        self.df['word_total'] = self.df.progress_apply(self.word_total, axis=1)

    @staticmethod
    def word_common(row):
        w1 = set(map(lambda word: word.lower().strip(), str(row['question1']).split()))
        w2 = set(map(lambda word: word.lower().strip(), str(row['question2']).split()))
        return 1.0 * len(w1 & w2)

    @staticmethod
    def word_share(row):
        w1 = set(map(lambda word: word.lower().strip(), str(row['question1']).split()))
        w2 = set(map(lambda word: word.lower().strip(), str(row['question2']).split()))
        return 1.0 * (len(w1 & w2) / (len(w1) + len(w2)))

    @staticmethod
    def word_total(row):
        w1 = set(map(lambda word: word.lower().strip(), str(row['question1']).split()))
        w2 = set(map(lambda word: word.lower().strip(), str(row['question2']).split()))
        return 1.0 * (len(w1) + len(w2))

# Assuming df is your DataFrame
feature_engineering_instance = FeatureEngineering(df)


100%|████████████████████████████████| 404290/404290 [00:04<00:00, 83802.85it/s]


In [6]:
from tqdm import tqdm

class FeatureEngineering:
    def __init__(self, dataframe):
        self.df = dataframe
        tqdm.pandas()
        self.calculate_frequency_features()
        self.calculate_length_and_word_features()

    def calculate_frequency_features(self):
        self.df['freq_qid1'] = self.df['qid1'].map(self.df['qid1'].value_counts())
        self.df['freq_qid2'] = self.df['qid2'].map(self.df['qid2'].value_counts())
        self.df['frq_q1+q2'] = self.df['freq_qid1'] + self.df['freq_qid2']
        self.df['frq_q1-q2'] = abs(self.df['freq_qid1'] - self.df['freq_qid2'])

    def calculate_length_and_word_features(self):
        self.df['q1len'] = self.df['question1'].str.len()
        self.df['q2len'] = self.df['question2'].str.len()
        self.df['q1_n_words'] = self.df['question1'].progress_apply(lambda row: len(str(row).split()))
        self.df['q2_n_words'] = self.df['question2'].progress_apply(lambda row: len(str(row).split()))
        self.df['word_common'] = self.df.progress_apply(self.word_common, axis=1)
        self.df['word_share'] = self.df.progress_apply(self.word_share, axis=1)
        self.df['word_total'] = self.df.progress_apply(self.word_total, axis=1)

    @staticmethod
    def word_common(row):
        w1 = set(map(lambda word: word.lower().strip(), str(row['question1']).split()))
        w2 = set(map(lambda word: word.lower().strip(), str(row['question2']).split()))
        return 1.0 * len(w1 & w2)

    @staticmethod
    def word_share(row):
        w1 = set(map(lambda word: word.lower().strip(), str(row['question1']).split()))
        w2 = set(map(lambda word: word.lower().strip(), str(row['question2']).split()))
        return 1.0 * (len(w1 & w2) / (len(w1) + len(w2)))

    @staticmethod
    def word_total(row):
        w1 = set(map(lambda word: word.lower().strip(), str(row['question1']).split()))
        w2 = set(map(lambda word: word.lower().strip(), str(row['question2']).split()))
        return 1.0 * (len(w1) + len(w2))

# Assuming df is your DataFrame
feature_engineering_instance = FeatureEngineering(df)


100%|████████████████████████████████| 404290/404290 [00:05<00:00, 77439.15it/s]


In [7]:
from tqdm import tqdm

class FeatureEngineering:
    def __init__(self, dataframe):
        self.df = dataframe
        tqdm.pandas()
        self.calculate_frequency_features()
        self.calculate_length_and_word_features()

    def calculate_frequency_features(self):
        self.df['freq_qid1'] = self.df['qid1'].map(self.df['qid1'].value_counts())
        self.df['freq_qid2'] = self.df['qid2'].map(self.df['qid2'].value_counts())
        self.df['frq_q1+q2'] = self.df['freq_qid1'] + self.df['freq_qid2']
        self.df['frq_q1-q2'] = abs(self.df['freq_qid1'] - self.df['freq_qid2'])

    def calculate_length_and_word_features(self):
        self.df['q1len'] = self.df['question1'].str.len()
        self.df['q2len'] = self.df['question2'].str.len()
        self.df['q1_n_words'] = self.df['question1'].progress_apply(lambda row: len(str(row).split()))
        self.df['q2_n_words'] = self.df['question2'].progress_apply(lambda row: len(str(row).split()))
        self.df['word_common'] = self.df.progress_apply(self.word_common, axis=1)
        self.df['word_share'] = self.df.progress_apply(self.word_share, axis=1)
        self.df['word_total'] = self.df.progress_apply(self.word_total, axis=1)

    @staticmethod
    def word_common(row):
        w1 = set(map(lambda word: word.lower().strip(), str(row['question1']).split()))
        w2 = set(map(lambda word: word.lower().strip(), str(row['question2']).split()))
        return 1.0 * len(w1 & w2)

    @staticmethod
    def word_share(row):
        w1 = set(map(lambda word: word.lower().strip(), str(row['question1']).split()))
        w2 = set(map(lambda word: word.lower().strip(), str(row['question2']).split()))
        return 1.0 * (len(w1 & w2) / (len(w1) + len(w2)))

    @staticmethod
    def word_total(row):
        w1 = set(map(lambda word: word.lower().strip(), str(row['question1']).split()))
        w2 = set(map(lambda word: word.lower().strip(), str(row['question2']).split()))
        return 1.0 * (len(w1) + len(w2))

# Assuming df is your DataFrame
feature_engineering_instance = FeatureEngineering(df)


100%|████████████████████████████████| 404290/404290 [00:05<00:00, 79957.07it/s]


In [8]:
df

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,frq_q1+q2,frq_q1-q2,q1len,q2len,q1_n_words,q2_n_words,word_common,word_share,word_total
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,1,1,2,0,66.0,57.0,14,12,10.0,0.434783,23.0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,4,1,5,3,51.0,88.0,8,13,4.0,0.200000,20.0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,1,1,2,0,73.0,59.0,14,10,4.0,0.166667,24.0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,1,1,2,0,50.0,65.0,11,9,0.0,0.000000,19.0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,3,1,4,2,76.0,39.0,13,7,2.0,0.100000,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0,2,2,4,0,85.0,79.0,14,13,11.0,0.440000,25.0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1,12,1,13,11,41.0,42.0,8,9,5.0,0.312500,16.0
404287,404287,537928,537929,What is one coin?,What's this coin?,0,1,1,2,0,17.0,17.0,4,3,1.0,0.142857,7.0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0,1,1,2,0,94.0,127.0,17,25,1.0,0.025000,40.0


# preprocceing the text

In [9]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

tqdm.pandas()

# Define the preprocessing function
def preprocess_text(text):
    if pd.isna(text):
        return text

    # Convert to lowercase
    text = text.lower()

    # Replace specific patterns
    replacements = {
        ",000,000": "m", ",000": "k", "′": "'", "’": "'", "won't": "will not",
        "cannot": "can not", "can't": "can not", "n't": " not", "what's": "what is",
        "it's": "it is", "'ve": " have", "i'm": "i am", "'re": " are", "he's": "he is",
        "she's": "she is", "'s": " own", "%": " percent ", "₹": " rupee ",
        "$": " dollar ", "€": " euro ", "'ll": " will", "...": " "
    }
    for key, value in replacements.items():
        text = text.replace(key, value)

    # Replace specific patterns using regular expressions
    text = re.sub(r"([0-9]+)000000", r"1m", text)
    text = re.sub(r"([0-9]+)000", r"1k", text)

    # Initialize a Porter stemmer
    porter = PorterStemmer()

    # Define a pattern to match non-alphanumeric characters
    pattern = re.compile('\W')

    # Remove non-alphanumeric characters
    text = re.sub(pattern, ' ', text)

    # Apply stemming using Porter Stemmer
    text = porter.stem(text)

    # Use BeautifulSoup to remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Remove duplicate words
    text = ' '.join(set(text.split()))

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])

    return text

# Apply the preprocessing function to 'question1' column with tqdm progress bar
df['question1'] = df['question1'].progress_apply(preprocess_text)

# Apply the preprocessing function to 'question2' column with tqdm progress bar
df['question2'] = df['question2'].progress_apply(preprocess_text)


100%|█████████████████████████████████| 404290/404290 [01:00<00:00, 6698.44it/s]


In [10]:
df

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,frq_q1+q2,frq_q1-q2,q1len,q2len,q1_n_words,q2_n_words,word_common,word_share,word_total
0,0,1,2,india share step guide market invest,share step guide market invest,0,1,1,2,0,66.0,57.0,14,12,10.0,0.434783,23.0
1,1,3,4,kohinoor story koh noor diamond,would happen stole kohinoor koh back noor diam...,0,4,1,5,3,51.0,88.0,8,13,4.0,0.200000,20.0
2,2,5,6,internet connection increase vpn speed using,internet dns increased hacking speed,0,1,1,2,0,73.0,59.0,14,10,4.0,0.166667,24.0
3,3,7,8,mentally solve lonely,24 math divided remainder find 23,0,1,1,2,0,50.0,65.0,11,9,0.0,0.000000,19.0
4,4,9,10,methane quikly oxide sugar carbon di dissolve ...,would survive water fish salt,0,3,1,4,2,76.0,39.0,13,7,2.0,0.100000,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404285,404285,433578,379845,keywords version language programming many lat...,keywords perl version language programming man...,0,2,2,4,0,85.0,79.0,14,13,11.0,0.440000,25.0
404286,404286,18840,155606,believe death life,true death life,1,12,1,13,11,41.0,42.0,8,9,5.0,0.312500,16.0
404287,404287,537928,537929,one coin,coin,0,1,1,2,0,17.0,17.0,4,3,1.0,0.142857,7.0
404288,404288,537930,537931,living approx uic chicago student cost annual ...,hairfall problem wax little hair clay gel prod...,0,1,1,2,0,94.0,127.0,17,25,1.0,0.025000,40.0


In [21]:
from tqdm import tqdm

class cleanFeatureEngineering:
    def __init__(self, dataframe):
        self.df = dataframe
        self.df.fillna(0, inplace=True)  # Fill all null values with 0
        tqdm.pandas()
        self.calculate_length_and_word_features()

    def calculate_length_and_word_features(self):
        self.df['c_q1len'] = self.df['question1'].str.len()
        self.df['c_q2len'] = self.df['question2'].str.len()
        self.df['c_q1_n_words'] = self.df['question1'].progress_apply(lambda row: len(str(row).split()))
        self.df['c_q2_n_words'] = self.df['question2'].progress_apply(lambda row: len(str(row).split()))
        self.df['c_word_common'] = self.df.progress_apply(self.word_common, axis=1)
        self.df['c_word_share'] = self.df.progress_apply(self.word_share, axis=1)
        self.df['c_word_total'] = self.df.progress_apply(self.word_total, axis=1)

    @staticmethod
    def word_common(row):
        w1 = set(map(lambda word: word.lower().strip(), str(row['question1']).split()))
        w2 = set(map(lambda word: word.lower().strip(), str(row['question2']).split()))
        return 1.0 * len(w1 & w2)

    @staticmethod
    def word_share(row):
        w1 = set(map(lambda word: word.lower().strip(), str(row['question1']).split()))
        w2 = set(map(lambda word: word.lower().strip(), str(row['question2']).split()))

        # Check if both sets are non-empty to avoid division by zero
        if len(w1) == 0 or len(w2) == 0:
            return 0.0
        else:
            return 1.0 * (len(w1 & w2) / (len(w1) + len(w2)))

    @staticmethod
    def word_total(row):
        w1 = set(map(lambda word: word.lower().strip(), str(row['question1']).split()))
        w2 = set(map(lambda word: word.lower().strip(), str(row['question2']).split()))
        return 1.0 * (len(w1) + len(w2))

# Assuming df is your DataFrame
c_feature_engineering_instance = cleanFeatureEngineering(df)


100%|████████████████████████████████| 404290/404290 [00:04<00:00, 94628.53it/s]


In [22]:
df

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,frq_q1+q2,frq_q1-q2,...,word_common,word_share,word_total,c_q1len,c_q2len,c_q1_n_words,c_q2_n_words,c_word_common,c_word_share,c_word_total
0,0,1,2,india share step guide market invest,share step guide market invest,0,1,1,2,0,...,10.0,0.434783,23.0,36.0,30.0,6,5,5.0,0.454545,11.0
1,1,3,4,kohinoor story koh noor diamond,would happen stole kohinoor koh back noor diam...,0,4,1,5,3,...,4.0,0.200000,20.0,31.0,67.0,5,10,4.0,0.266667,15.0
2,2,5,6,internet connection increase vpn speed using,internet dns increased hacking speed,0,1,1,2,0,...,4.0,0.166667,24.0,44.0,36.0,6,5,2.0,0.181818,11.0
3,3,7,8,mentally solve lonely,24 math divided remainder find 23,0,1,1,2,0,...,0.0,0.000000,19.0,21.0,33.0,3,6,0.0,0.000000,9.0
4,4,9,10,methane quikly oxide sugar carbon di dissolve ...,would survive water fish salt,0,3,1,4,2,...,2.0,0.100000,20.0,60.0,29.0,10,5,2.0,0.133333,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404285,404285,433578,379845,keywords version language programming many lat...,keywords perl version language programming man...,0,2,2,4,0,...,11.0,0.440000,25.0,56.0,54.0,7,7,6.0,0.428571,14.0
404286,404286,18840,155606,believe death life,true death life,1,12,1,13,11,...,5.0,0.312500,16.0,18.0,15.0,3,3,2.0,0.333333,6.0
404287,404287,537928,537929,one coin,coin,0,1,1,2,0,...,1.0,0.142857,7.0,8.0,4.0,2,1,1.0,0.333333,3.0
404288,404288,537930,537931,living approx uic chicago student cost annual ...,hairfall problem wax little hair clay gel prod...,0,1,1,2,0,...,1.0,0.025000,40.0,61.0,77.0,9,13,0.0,0.000000,22.0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             404290 non-null  int64  
 1   qid1           404290 non-null  int64  
 2   qid2           404290 non-null  int64  
 3   question1      404290 non-null  object 
 4   question2      404290 non-null  object 
 5   is_duplicate   404290 non-null  int64  
 6   freq_qid1      404290 non-null  int64  
 7   freq_qid2      404290 non-null  int64  
 8   frq_q1+q2      404290 non-null  int64  
 9   frq_q1-q2      404290 non-null  int64  
 10  q1len          404290 non-null  float64
 11  q2len          404290 non-null  float64
 12  q1_n_words     404290 non-null  int64  
 13  q2_n_words     404290 non-null  int64  
 14  word_common    404290 non-null  float64
 15  word_share     404290 non-null  float64
 16  word_total     404290 non-null  float64
 17  c_q1len        404289 non-nul

In [24]:
df.shape

(404290, 24)

In [25]:
clean_df=df.drop(['question1','question2','id','qid1','qid2'],axis=1)

In [30]:
clean_df.fillna(0, inplace=True)  # Fill all null values with 0


In [31]:
df.describe()

,id,qid1,qid2,is_duplicate,freq_qid1,freq_qid2,frq_q1+q2,frq_q1-q2,q1len,q2len,...,word_common,word_share,word_total,c_q1len,c_q2len,c_q1_n_words,c_q2_n_words,c_word_common,c_word_share,c_word_total
count,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000,...,404290.000000,404290.000000,404290.000000,404289.000000,404288.000000,404290.000000,404290.000000,404290.000000,404290.000000,404290.000000
mean,202144.500000,217243.942418,220955.655337,0.369198,2.827609,3.046961,5.874570,1.845715,59.536709,60.108365,...,4.511586,0.223401,21.121457,36.061201,35.983208,5.389671,5.442348,2.514574,0.253197,10.832019
std,116708.614502,157751.700002,159903.182629,0.482588,4.487418,6.026871,9.091068,5.187882,29.940655,33.863870,...,3.098622,0.128351,8.588812,19.493611,20.843971,2.795895,3.031648,1.709875,0.140748,5.095336
min,0.000000,1.000000,2.000000,0.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,101072.250000,74437.500000,74727.000000,0.000000,1.000000,1.000000,2.000000,0.000000,39.000000,39.000000,...,2.000000,0.125000,15.000000,22.000000,22.000000,3.000000,3.000000,2.000000,0.153846,7.000000
50%,202144.500000,192182.000000,197052.000000,0.000000,1.000000,1.000000,3.000000,0.000000,52.000000,51.000000,...,4.000000,0.217391,19.000000,31.000000,31.000000,5.000000,5.000000,2.000000,0.250000,10.000000
75%,303216.750000,346573.500000,354692.500000,1.000000,3.000000,2.000000,5.000000,2.000000,72.000000,72.000000,...,6.000000,0.318182,25.000000,45.000000,44.000000,7.000000,7.000000,3.000000,0.363636,13.000000
max,404289.000000,537932.000000,537933.000000,1.000000,50.000000,120.000000,157.000000,119.000000,623.000000,1169.000000,...,41.000000,0.500000,166.000000,327.000000,556.000000,52.000000,85.000000,24.000000,0.500000,99.000000


In [32]:
clean_df.isnull().sum()

is_duplicate     0
freq_qid1        0
freq_qid2        0
frq_q1+q2        0
frq_q1-q2        0
q1len            0
q2len            0
q1_n_words       0
q2_n_words       0
word_common      0
word_share       0
word_total       0
c_q1len          0
c_q2len          0
c_q1_n_words     0
c_q2_n_words     0
c_word_common    0
c_word_share     0
c_word_total     0
dtype: int64

In [34]:
# train text split
X = clean_df.drop('is_duplicate',axis=1)
y =  clean_df['is_duplicate']

In [35]:
X

,freq_qid1,freq_qid2,frq_q1+q2,frq_q1-q2,q1len,q2len,q1_n_words,q2_n_words,word_common,word_share,word_total,c_q1len,c_q2len,c_q1_n_words,c_q2_n_words,c_word_common,c_word_share,c_word_total
0,1,1,2,0,66.0,57.0,14,12,10.0,0.434783,23.0,36.0,30.0,6,5,5.0,0.454545,11.0
1,4,1,5,3,51.0,88.0,8,13,4.0,0.200000,20.0,31.0,67.0,5,10,4.0,0.266667,15.0
2,1,1,2,0,73.0,59.0,14,10,4.0,0.166667,24.0,44.0,36.0,6,5,2.0,0.181818,11.0
3,1,1,2,0,50.0,65.0,11,9,0.0,0.000000,19.0,21.0,33.0,3,6,0.0,0.000000,9.0
4,3,1,4,2,76.0,39.0,13,7,2.0,0.100000,20.0,60.0,29.0,10,5,2.0,0.133333,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404285,2,2,4,0,85.0,79.0,14,13,11.0,0.440000,25.0,56.0,54.0,7,7,6.0,0.428571,14.0
404286,12,1,13,11,41.0,42.0,8,9,5.0,0.312500,16.0,18.0,15.0,3,3,2.0,0.333333,6.0
404287,1,1,2,0,17.0,17.0,4,3,1.0,0.142857,7.0,8.0,4.0,2,1,1.0,0.333333,3.0
404288,1,1,2,0,94.0,127.0,17,25,1.0,0.025000,40.0,61.0,77.0,9,13,0.0,0.000000,22.0


In [36]:
from sklearn.model_selection import train_test_split

In [40]:
xtrain,xtest,ytrain,ytest=train_test_split(X,y,random_state=42,test_size=0.30)

In [41]:
xtrain.shape

(283003, 18)

In [42]:
xtest.shape

(121287, 18)

In [43]:
ytrain.shape

(283003,)

In [44]:
ytest.shape

(121287,)